In [83]:
from numpy import dot
from numpy.linalg import norm
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [84]:
merged = pd.read_pickle("../Data/merged.pkl")

In [46]:
features = list(merged.columns)
for feature in features:
    merged[feature] = (merged[feature] - merged[feature].mean())/merged[feature].std()

In [85]:
merged

,Автотранспорт,Алкогольный магазин,"Бар ""Иной""","Бар ""Коктейль""",Барбершоп,Не определено,Перевод денег на депозит,Покупка одежды,Покупка техники,Покупки Ozon,"Ресторан ""Буйвол""",Салон красоты,Такси,Турагенство,Сумма расходов
ID Клиента,,,,,,,,,,,,,,,
10157447,2,16,0,11,26,17,0,2,0,0,11,0,9,13,-0.259226
10157454,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-0.269286
10158439,0,0,0,9,11,0,0,0,5,0,2,0,0,0,-0.228334
10158854,6,44,0,61,56,44,0,2,53,7,95,5,89,66,-0.218408
10159585,6,4,0,2,0,11,0,3,3,8,0,3,1,26,-0.263764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11328050,0,9,0,37,15,27,0,4,29,2,15,0,19,29,-0.193631
11328121,0,2,0,0,0,2,0,4,0,0,0,0,5,0,-0.046777
11335636,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.279330


In [86]:
# Посещаемость
ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in [10359664,11007017,10356174,10361741,10562323,11128132,10358256,10830303,10216655,11125776]]

In [87]:
ideals

[[7.0,
  78.0,
  0.0,
  115.0,
  120.0,
  70.0,
  0.0,
  55.0,
  34.0,
  20.0,
  121.0,
  6.0,
  43.0,
  15.0,
  -0.25028400118536076],
 [0.0,
  22.0,
  0.0,
  14.0,
  26.0,
  13.0,
  0.0,
  0.0,
  0.0,
  12.0,
  30.0,
  1.0,
  2.0,
  25.0,
  -0.2368111909955255],
 [4.0,
  11.0,
  2.0,
  81.0,
  113.0,
  126.0,
  0.0,
  13.0,
  12.0,
  0.0,
  81.0,
  0.0,
  8.0,
  28.0,
  -0.20801244617736755],
 [2.0,
  8.0,
  0.0,
  48.0,
  68.0,
  34.0,
  2.0,
  6.0,
  9.0,
  9.0,
  31.0,
  8.0,
  10.0,
  28.0,
  -0.2580292906626685],
 [0.0,
  2.0,
  0.0,
  6.0,
  31.0,
  0.0,
  0.0,
  5.0,
  0.0,
  0.0,
  2.0,
  2.0,
  0.0,
  82.0,
  -0.12366200505385151],
 [0.0,
  4.0,
  0.0,
  28.0,
  28.0,
  14.0,
  0.0,
  0.0,
  1.0,
  4.0,
  10.0,
  0.0,
  14.0,
  8.0,
  -0.13921739081775256],
 [3.0,
  34.0,
  0.0,
  15.0,
  95.0,
  13.0,
  0.0,
  5.0,
  8.0,
  15.0,
  9.0,
  0.0,
  20.0,
  14.0,
  -0.21121600439853352],
 [7.0,
  59.0,
  0.0,
  39.0,
  138.0,
  26.0,
  0.0,
  16.0,
  25.0,
  30.0,
  56.0,
  0.0

In [11]:
def Norm(sp):
    mean = sum(sp[:-1]) / len(sp[:-1])
    variance = sum([((x - mean) ** 2) for x in sp[:-1]]) / len(sp[:-1])
    std = variance ** 0.5
    res = list(map(lambda x: (x - mean)/std, sp[:-1]))
    return res

In [20]:
print(Norm(ideals[0][:-1]))

[-0.9607796839552049, 0.6689387560643748, -1.1214561498726283, 1.5182286473421838, 1.6329975515689148, 0.4853085093016053, -1.1214561498726283, 0.1410017966214124, -0.34102760113085767, -0.6623805329657043, 1.655951332414261, -0.983733464800551, -0.13444357352274192, -0.7771494371924353]


In [89]:
# #Доход
# ideals2 = [list(map(float,merged2020.loc[i].to_numpy(dtype=float))) for i in [10409000,11083894,10356174,10361741,10562323,11128132]]

In [90]:
merge_vec = list(map(list,merged.to_numpy(dtype=float)))

In [91]:
-0.21138157369570007

-0.21138157369570007

In [93]:
clients = {key:val for val,key in zip(merge_vec,merged.index.values.tolist())}

In [97]:
class HybridRecommender:
    def __init__(self, ideal):
        self.clients_vector = None
        self.ideal_clients = ideal
          
    def cos_sim(self,a,b):
        similarity = dot(a, b)/(norm(a)*norm(b))
        return similarity
    
    def collaborative_filtering(self,user):
        count = 0
        for i in self.ideal_clients:
            similarity = self.cos_sim(i,user)
            print(similarity)
            if 0.9 <= similarity < 1:
                count += 1
        return count >= 2
        
    def content_filtering(self):
        pass
#         cv = CountVectorizer()
#         movie_matrix = cv.fit_transform(self.movies_df['genres'])
#         movie_similarities = cosine_similarity(movie_matrix)
#         self.movie_predictions = pd.DataFrame(movie_similarities.dot(self.movie_features), index=self.movies_df.index, columns=self.movie_features.columns)

    def hybrid_filtering(self):
        self.hybrid_predictions = self.user_predictions.add(self.movie_predictions, fill_value=0)

    def recommend_clients(self, clients):
        valid_clients = []
        for client_id, client in clients.items():
            if self.collaborative_filtering(client):
                valid_clients.append(client_id)
        return valid_clients



In [98]:
recommender = HybridRecommender(ideals)
Clients = recommender.recommend_clients(clients)

0.8719635497402806
0.8847699230136769
0.8720589769756846
0.907270806676814
0.5478943194682346
0.8709141753544308
0.8686437191710352
0.8422952175762861
0.6022961895205602
0.8258006738522499
0.16978810260539134
0.035232921316595014
0.0374831368089925
0.09597694273993408
0.0003650679277720117
0.2918506697612382
0.1800115061144148
0.5615928099626151
0.4171170519632547
0.3639622623403669
0.7432304201499081
0.5490676129953429
0.6955180711274751
0.8358047989932292
0.2980391517411578
0.8287002053957168
0.7570441918947237
0.6495088642999662
0.3979835504563643
0.6433058513135606
0.8447581033805082
0.8148900888791161
0.7277853062529829
0.7851533879200929
0.47439454136340614
0.7915393396279563
0.6174164072161187
0.8324037686057392
0.7387577212625185
0.8683446423114505
0.2964391008704337
0.5809054012313465
0.3804215333498408
0.4473030621141033
0.8025258668320122
0.33831498409912963
0.2630594060468156
0.255428797819939
0.504982186233507
0.5514713558286358
0.6349481336159354
0.4246241198959858
0.5887

In [99]:
print(len(Clients))
Clients

38


[10206023,
 10214586,
 10216816,
 10217384,
 10218864,
 10354725,
 10355392,
 10355417,
 10356174,
 10360759,
 10361741,
 10363026,
 10397820,
 10404555,
 10405540,
 10406690,
 10409000,
 10574677,
 10577475,
 10580277,
 10580935,
 10583558,
 10831118,
 10837991,
 10838002,
 11000129,
 11008961,
 11018939,
 11099056,
 11121022,
 11135326,
 11255812,
 11257217,
 11257902,
 11259088,
 11259157,
 11259771,
 11259804]